In [46]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import tweepy
import re 
from tweepy import OAuthHandler 
from textblob import TextBlob
from nltk.corpus import stopwords
import requests
from bs4 import BeautifulSoup

#cursor
#tweet.text

In [48]:
url = 'https://www.imdb.com/chart/tvmeter?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=4da9d9a5-d299-43f2-9c53-f0efa18182cd&pf_rd_r=0KYJP3BJH5DN6EZWRQ5Z&pf_rd_s=right-4&pf_rd_t=15506&pf_rd_i=toptv&ref_=chttvtp_ql_5'

In [49]:
soup = requests.get(url).content
soup = BeautifulSoup(soup,'html.parser')

In [53]:
#Las series más populares
popularTV = soup.select("td.titleColumn a[href^='/title']")
popularTV = [name.text for name in popularTV]
popularTV = list(map(lambda x:x.strip(),popularTV))
top10 = popularTV[:11]

In [54]:
#Score
scores = soup.select("td.ratingColumn strong")
score = [score.text for score in scores]
top10score = score[:11]

In [80]:
#Crear el dataframe de las series más populares en IMDB
columns = ["Name", "Score"]
dicttop = {'Name':top10,'Score':top10score}

top10df = pd.DataFrame(dicttop, columns=columns)
top10df

,Name,Score
0,Watchmen,6.8
1,Cómo vivir contigo mismo,7.4
2,Peaky Blinders,8.8
3,The Walking Dead,8.3
4,Modern Love,8.1
5,American Horror Story,8.1
6,Breaking Bad,9.5
7,Daybreak,6.8
8,Batwoman,3.2
9,Supernatural,8.4


In [7]:
#Acceder a API Twitter
API_KEY = "CRuEovkPCCTmYVAC8PbndD9N3"
API_SECRET = "SikynKdtF2NpEDpx9xvdtuS9wMHbgQXFAyCq5qj1HYbCJfnhEZ"
ACCESS_TOKEN = "79345899-q0Yb2A2BOfNXkNEjqdFU474r8yARK2WkUqhlTYDjY"
ACCESS_TOKEN_SECRET = "OUD4HI3o11cc4VC2TkTcxSYWTQAcjsh2mlni3SGlkA3fy"

auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [101]:
#Buscar los mejores tweets de una serie
best_tweets = api.search(q='Watchmen', lang = 'en', count = 100, result_type='mixed')



In [102]:
#Pasar el api.search a df
def get_text(json):
    texts = [pd.Series(best_tweet._json) for best_tweet in json]
    df = pd.DataFrame(texts)
    return df
tweets = get_text(best_tweets)
tweets

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,...,favorite_count,favorited,retweeted,possibly_sensitive,lang,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status,extended_entities
0,Fri Nov 01 17:00:06 +0000 2019,1190312570244214784,1190312570244214784,Very excited to announce The Official @Watchme...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'result_type': 'popular', 'iso_language_code'...","<a href=""https://twittimer.com"" rel=""nofollow""...",NaN,None,...,1452,False,False,False,en,NaN,NaN,NaN,NaN,NaN
1,Fri Nov 01 17:00:00 +0000 2019,1190312545510477825,1190312545510477825,The Official Watchmen Podcast is coming. Join ...,True,"{'hashtags': [{'text': 'Chernobyl', 'indices':...","{'result_type': 'popular', 'iso_language_code'...","<a href=""https://studio.twitter.com"" rel=""nofo...",NaN,None,...,544,False,False,False,en,NaN,NaN,NaN,NaN,NaN
2,Fri Nov 01 15:15:31 +0000 2019,1190286251510902787,1190286251510902787,white comic fans hate watchmen. but black woma...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'result_type': 'popular', 'iso_language_code'...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,401,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN
3,Sat Nov 02 23:27:42 +0000 2019,1190772503104413696,1190772503104413696,@ProperOpinion Thought the same about the watc...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",1.190757e+18,1190757443216785409,...,1,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN
4,Sat Nov 02 23:27:42 +0000 2019,1190772500075929600,1190772500075929600,@ReginaKing @HBO You’re awesome in The Leftove...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",1.190723e+18,1190722756511334400,...,0,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Sat Nov 02 22:59:19 +0000 2019,1190765360116621320,1190765360116621320,RT @michaelharriot: Thread:\n\nA lot of white ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,None,...,0,False,False,NaN,en,{'created_at': 'Tue Oct 22 02:24:21 +0000 2019...,NaN,NaN,NaN,NaN
96,Sat Nov 02 22:59:06 +0000 2019,1190765305041211392,1190765305041211392,RT @tvs_movies: #TheBatman: William Hoy Film E...,False,"{'hashtags': [{'text': 'TheBatman', 'indices':...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,0,False,False,NaN,en,{'created_at': 'Sat Nov 02 12:50:53 +0000 2019...,NaN,NaN,NaN,NaN
97,Sat Nov 02 22:58:38 +0000 2019,1190765187516829696,1190765187516829696,RT @tvs_movies: #TheBatman: William Hoy Film E...,False,"{'hashtags': [{'text': 'TheBatman', 'indices':...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,0,False,False,NaN,en,{'created_at': 'Sat Nov 02 12:50:53 +0000 2019...,NaN,NaN,NaN,NaN
98,Sat Nov 02 22:58:18 +0000 2019,1190765103978893312,1190765103978893312,The End Is the Beginning Is the End https://t....,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,0,False,False,False,en,NaN,NaN,NaN,NaN,NaN


In [103]:
#Dejar solo el texto
text = tweets['text']

In [104]:
#Convertirlo a dataframe
tweetsdf= pd.DataFrame(text)
tweetsdf

,text
0,Very excited to announce The Official @Watchme...
1,The Official Watchmen Podcast is coming. Join ...
2,white comic fans hate watchmen. but black woma...
3,@ProperOpinion Thought the same about the watc...
4,@ReginaKing @HBO You’re awesome in The Leftove...
...,...
95,RT @michaelharriot: Thread:\n\nA lot of white ...
96,RT @tvs_movies: #TheBatman: William Hoy Film E...
97,RT @tvs_movies: #TheBatman: William Hoy Film E...
98,The End Is the Beginning Is the End https://t....


In [81]:
#función para clasificar tweets
def get_tweet_sentiment(tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(tweet) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'

In [95]:
#Aplicar tweets a todo
sentiments = tweetsdf['text'].apply(get_tweet_sentiment)

In [100]:
tweetsdf['sentiments'] = sentiments
tweetsdf

,text,Sentiments,sentiments
0,Very excited to announce The Official @Watchme...,positive,positive
1,The Official Watchmen Podcast is coming. Join ...,neutral,neutral
2,white comic fans hate watchmen. but black woma...,negative,negative
3,RT @verge: What DC Comics needs to learn from ...,negative,negative
4,RT @SnyderContent: Zack Snyder's #Watchmen: Th...,neutral,neutral
5,I have almost zero clue what is happening on W...,positive,positive
6,@firststartrmh I think I shall watch Xmas Rob ...,positive,positive
7,@megankatenelson @watchmen I am! I have a lot...,neutral,neutral
8,A must watch for anyone watching #Watchmen \na...,positive,positive
9,@ReginaKing @HBO I love this show more than my...,positive,positive


In [ ]:
def main(): 
    # creating object of TwitterClient Class 
    api = TwitterClient() 
    # calling function to get tweets 
    tweets = api.get_tweets(query = 'Donald Trump', count = 200) 
  
    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # percentage of positive tweets 
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    # percentage of negative tweets 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
    # percentage of neutral tweets 
    print("Neutral tweets percentage: {} % \ 
        ".format(100*len(tweets - ntweets - ptweets)/len(tweets))) 
  
    # printing first 5 positive tweets 
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:10]: 
        print(tweet['text']) 
  
    # printing first 5 negative tweets 
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:10]: 
        print(tweet['text']) 
  
if __name__ == "__main__": 
    # calling main function 
    main() 

In [ ]:
tweets = api.get_tweets(query = 'Donald Trump', count = 200)